# Zadatak 1. Učitavanje podataka (25% bodova)

In [1]:
import prvi
import torch

Sve metode potrebne za prvi zadatak nalaze se u datoteci prvi.py. U sljedećem isječku koda računamo frekvencije pojavljivanja riječi, stvaramo vokabulare i učitavamo dataset.

In [2]:
freq_data, freq_label = prvi.get_frequencies('data/sst_train_raw.csv')
text_vocab = prvi.Vocab(freq_data, max_size=-1, min_freq=1)
label_vocab = prvi.Vocab(freq_label, use_extra=False)
train_dataset = prvi.NLPDataset('data/sst_train_raw.csv', text_vocab, label_vocab)

## Razred Vocab

Vaša implementacija razreda Vocab mora implementirati funkcionalnost pretvorbe niza tokena (ili jednog tokena) u brojeve. Ovu funkciju možete nazvati encode. Primjer ove pretvorbe za četvrtu instancu train skupa:

In [3]:
instance_text, instance_label = train_dataset.instances[3]
print(f"Text: {instance_text}")
print(f"Label: {instance_label}")
print(f"Numericalized text: {text_vocab.encode(instance_text)}")
print(f"Numericalized label: {label_vocab.encode(instance_label)}")

Text: ['yet', 'the', 'act', 'is', 'still', 'charming', 'here']
Label: positive
Numericalized text: tensor([189,   2, 674,   7, 129, 348, 143])
Numericalized label: tensor([0])


Također, vaša implementacija razreda Vocab mora primati iduće parametre:

max_size: maksimalni broj tokena koji se sprema u vokabular (uključuje i posebne znakove). -1 označava da se spremaju svi tokeni.
min_freq: minimalna frekvencija koju token mora imati da bi ga se spremilo u vokabular (\ge). Posebni znakovi ne prolaze ovu provjeru.
Primjer izgradnje vokabulara sa svim tokenima (duljina uključuje i posebne znakove):

In [4]:
text_vocab = prvi.Vocab(freq_data, max_size=-1, min_freq=0)
print(len(text_vocab.itos))

14806


## Učitavanje vektorskih reprezentacija

Vaš zadatak je implementirati funkciju koja će za zadani vokabular (iterable stringova) generirati embedding matricu. Vaša funkcija treba podržavati dva načina genriranja embedding matrice: nasumična inicijalizacija iz standardne normalne razdiobe (N(0,1)) i učitavanjem iz datoteke. Pri učitavanju iz datoteke, ako ne pronađete vektorsku reprezentaciju za neku riječ, inicijalizirajte ju normalno. Vektorsku reprezentaciju za znak punjenja (na indeksu 0) morate inicijalizirati na vektor nula. Jednostavan način na koji možete implementirati ovo učitavanje je da inicijalirate matricu iz standardne normalne razdiobe, a potom prebrišete inicijalnu reprezentaciju u retku za svaku riječ koju učitate. Bitno: Pripazite da redoslijed vektorskih reprezentacija u matrici odgovara redoslijedu riječi u vokabularu! Npr., na indeksu 0 mora biti reprezentacija za posebni znak punjenja.

In [5]:
embedding_matrix = prvi.get_embedding_matrix(text_vocab, 'data/sst_glove_6b_300d.txt', 300)
print(embedding_matrix.shape)

torch.Size([14806, 300])


Jednom kad ste uspješno učitali vašu V×d embedding matricu, iskoristite torch.nn.Embedding.from_pretrained() kako bi vašu matricu spremili u optimizirani omotač za vektorske reprezentacije. Postavite parametar funkcije padding_idx na 0 (indeks znaka punjenja u vašoj embedding matrici), a parametar funkcije freeze ostavite na True ako koristite predtrenirane reprezentacije, a postavite na False inače.

## Nadjačavanje metoda torch.utils.data.Dataset

Da bi naša implementacija razreda NLPDataset bila potpuna, potrebno je nadjačati __getitem__ metodu koja omogućava indeksiranje razreda. Za potrebe vježbe, ta metoda treba vraćati numerikalizirani text i labelu referencirane instance. Također, dovoljno je napraviti da se numerikalizacija radi “on-the-fly”, i nije ju nužno cachirati.

Primjer numerikalizacije s nadjačavanjem:

In [6]:
instance_text, instance_label = train_dataset.instances[3]
print(f"Text: {instance_text}")
print(f"Label: {instance_label}")
numericalized_text, numericalized_label = train_dataset[3]
print(f"Numericalized text: {numericalized_text}")
print(f"Numericalized label: {numericalized_label}")

Text: ['yet', 'the', 'act', 'is', 'still', 'charming', 'here']
Label: positive
Numericalized text: tensor([189,   2, 674,   7, 129, 348, 143])
Numericalized label: tensor([0])


# Implementacija batchiranja podataka: collate funkcija

Zadatak naše collate funkcije biti će nadopuniti duljine instanci znakom punjenja do duljine najdulje instance u batchu. Za ovo, pogledajte funkciju from torch.nn.utils.rnn.pad_sequence. Primjetite da vaša implementacija collate funkcije mora znati koji se indeks koristi kao znak punjenja.

Jednom kad smo implementirali sve navedeno, naše učitavanje podataka bi moglo izgledati ovako:

In [7]:
batch_size = 2 # Only for demonstrative purposes
shuffle = False # Only for demonstrative purposes
train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, 
                              shuffle=shuffle, collate_fn=prvi.pad_collate_fn)
texts, labels, lengths = next(iter(train_data_loader))
print(f"Texts: {texts}")
print(f"Labels: {labels}")
print(f"Lengths: {lengths}")

Texts: tensor([[   2,  554,    7, 2872,    6,   22,    2, 2873, 1236,    8,   96, 4800,
            4,   10,   72,    8,  242,    6,   75,    3, 3576,   56, 3577,   34,
         2022, 2874, 7123, 3578, 7124,   42,  779, 7125,    0,    0],
        [   2, 2875, 2023, 4801,    5,    2, 3579,    5,    2, 2876, 4802,    7,
           40,  829,   10,    3, 4803,    5,  627,   62,   27, 2877, 2024, 4804,
          962,  715,    8, 7126,  555,    5, 7127, 4805,    8, 7128]])
Labels: tensor([0, 0])
Lengths: tensor([32, 34])


# Zadatak 2. Implementacija baseline modela (25% bodova)

Prvi korak kod svakog zadatka strojnog učenja bi trebao biti implementacija baseline modela. Baseline model nam služi za procjenu performansi koje naš stvarni, uobičajeno skuplji model mora moći preći kao plitak potok. Također, baseline modeli će nam pokazati kolika je stvarno cijena izvođenja naprednijih modela.

Vaš zadatak u laboratorijskoj vježbi je implementirati model koji će koristiti sažimanje usrednjavanjem (eng. mean pooling) kako bi eliminirao problematičnu varijabilnu dimenziju. Pri primjeni sažimanja usrednjavanjem odmah eliminirajte cijelu vremensku dimenziju (tzv. okno je veličine T).

Osnovni model koji implementirate mora izgledati ovako:

    avg_pool() -> fc(300, 150) -> ReLU() -> fc(150, 150) -> ReLU() -> fc(150,1)
    
Kao gubitak predlažemo da koristite BCEWithLogitsLoss, u kojem slučaju ne morate primjeniti sigmoidu na izlaznim logitima. Alternativno, možete staviti da vam je izlazna dimenzionalnost broj klasa te koristiti gubitak unakrsne entropije. Oba pristupa su korištena u praksi ovisno o osobnim preferencama.

Kao algoritam optimizacije koristite Adam.

Implementirajte metrike praćenja performansi modela. Osim gubitka na skupu podataka, zanimaju nas preciznost (eng. accuracy), f1 mjera i matrica zabune (eng. confusion matrix). Nakon svake epohe ispišite performanse modela po svim metrikama na skupu za validaciju, a nakon zadnje epohe ispišite performanse modela na skupu za testiranje.

In [8]:
import drugi
import numpy as np

freq_data, freq_label = prvi.get_frequencies('data/sst_train_raw.csv')
text_vocab = prvi.Vocab(freq_data, max_size=-1, min_freq=1)
label_vocab = prvi.Vocab(freq_label, use_extra=False)
train_dataset = prvi.NLPDataset('data/sst_train_raw.csv', text_vocab, label_vocab)
embedding_matrix = prvi.get_embedding_matrix(text_vocab, 'data/sst_glove_6b_300d.txt', 300)

seed=7052020
torch.manual_seed(seed)
np.random.seed(seed)

model = drugi.BaselineModel(embedding_matrix)
model.train_model(epochs=5, dataset=train_dataset, optimizer=torch.optim.Adam(model.get_params(), lr=0.001), batch_size=10, text_vocab=text_vocab, label_vocab=label_vocab, verbose=True)

Epoch 1 results on validation dataset:
Accuracy: 0.6578802855573861
Precision: 0.6189300411522634
F1 score: 0.7070992007522331
Loss: 19.809267
Confusion matrix:
 [[446 463]
 [160 752]]

Epoch 2 results on validation dataset:
Accuracy: 0.6749038989566173
Precision: 0.6568627450980392
F1 score: 0.6935817805383023
Loss: 13.55767
Confusion matrix:
 [[559 350]
 [242 670]]

Epoch 3 results on validation dataset:
Accuracy: 0.6891817682591982
Precision: 0.6555755395683454
F1 score: 0.7203557312252964
Loss: 12.266848
Confusion matrix:
 [[526 383]
 [183 729]]

Epoch 4 results on validation dataset:
Accuracy: 0.6743547501372872
Precision: 0.627906976744186
F1 score: 0.7253358036127837
Loss: 12.924502
Confusion matrix:
 [[445 464]
 [129 783]]

Epoch 5 results on validation dataset:
Accuracy: 0.6749038989566173
Precision: 0.650093808630394
F1 score: 0.7007077856420626
Loss: 11.843177
Confusion matrix:
 [[536 373]
 [219 693]]

Results on test dataset:
Accuracy: 0.6892201834862385
Precision: 0.661190

# Zadatak 3. Implementacija povratne neuronske mreže (25% bodova)

Nakon što ste uspješno implementirali vaš baseline model, vrijeme je da isprobamo neki model baziran na povratnim neuronskim mrežama. Vaš zadatak je implementirati osnovni model povratne neuronske meže po izboru. Na izboru su vam iduće ćelije: [“Vanilla” RNN, GRU, LSTM].

Za odabrani model, detaljno pročitajte njegovu dokumentaciju. U nastavku ćemo vam samo skrenuti pozornost na nekoliko bitnih detalja:

- Svaka RNN mreža kao izlaz svoje forward metode vraća (1) niz skrivenih stanja posljednjeg sloja i (2) skriveno stanje (tj., skrivena stanja u slučaju LSTMa) za sve slojeve zadnjeg vremenskog koraka. Kao ulaz u dekoder obično želite staviti skriveno stanje iz zadnjeg sloja u zadnjem vremenskom koraku. Kod LSTMa, to je h komponenta dualnog (h, c) skrivenog stanja.

- Radi brzine, RNN mreže preferiraju inpute u time-first formatu (budući da je brže iterirati po prvoj dimenziji tenzora). Transponirajte ulaze prije nego ih šaljete RNN ćeliji.

- Tenzori koji su ulaz u RNN ćelije se često “pakiraju”. Pakiranje je zapis tenzora kojemu su pridružene stvarne duljine svakog elementa u batchu. Ako koristite pakiranje, RNN mreža se neće odmatati za vremenske korake koji sadrže padding u elementima batcha. Ovdje osim efikasnosti možete dobiti i na preciznosti, ali ovaj dio nije nužan dio vaše implementacije.
Implementirajte gradient clipping prije optimizacijskog koraka 

- Osnovni model vaše odabrane RNN ćelije treba izgledati ovako:

    rnn(150) -> rnn(150) -> fc(150, 150) -> ReLU() -> fc(150,1)
    
Vaš osnovni model RNN ćelije bi trebao biti jednosmjeran i imati dva sloja. Za višeslojni RNN iskoristite argument num_layers pri konstrukciji RNN mreže.